In [1]:
import geopandas as gpd
import dask_geopandas as dgpd
import pygeos
from dask.distributed import Client

# Enable Pygeos
gpd.options.use_pygeos = True


def load_and_simplify(filepath, tolerance=0.01):
    gdf = gpd.read_file(filepath)
    gdf['geometry'] = gdf['geometry'].simplify(tolerance)
    return gdf

def create_spatial_index(geoseries):
    return geoseries.sindex

def overlay_with_dask(gdf1, gdf2, how='intersection', npartitions=10):
    # Ensure that both GeoDataFrames have the same CRS
    if gdf1.crs != gdf2.crs:
        gdf2 = gdf2.to_crs(gdf1.crs)

    # Convert to Dask GeoDataFrames
    dask_gdf1 = dgpd.from_geopandas(gdf1, npartitions=npartitions)
    dask_gdf2 = dgpd.from_geopandas(gdf2, npartitions=npartitions)
    
    # Perform the overlay operation
    result = dask_gdf1.overlay(dask_gdf2, how=how).compute()
    
    return result


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\63\ipykernel_39972\113520683.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:

In [8]:
# Load required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import fiona
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from shapely.ops import unary_union
from unidecode import unidecode
import glob
import csv
from datetime import datetime
import dask.dataframe as dd
import dask_geopandas as dg
from dask.distributed import Client
import gc

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
outdata_f = os.path.join(indata_f, 'OutputData', 'test0307')
if not os.path.exists(outdata_f):
    # Create the folder if it doesn't exist
    os.makedirs(outdata_f)


# 0 PREPARE A LOG FILE FOR QC
log_file = 'log_GQA_Step1.csv'
log_path = os.path.join(outdata_f, log_file)

# Initialize Dask client
client = Client()

# 1 READ URBAN CENTRES
# Read shapefile
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG.shp')
# Read the GeoPackage file
uc = gpd.read_file(uc_file_path)


# 2 READ NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')

# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery')

cities_ls = ['Kaunas', 'København', 'Girona', 'Bordeaux']
# Loop through test cities
for cityLocalName in cities_ls[:1]:
    print(str(cityLocalName))
    start_time = datetime.now()
    print(str(start_time))

    uc_city = uc.query(f'HDENS_NAME == "{cityLocalName}"')
    ctry_code = uc_city.CNTR_CODE.values.astype(str)[0]
    cityLocalName_unicode = unidecode(cityLocalName)
    
    output_path = os.path.join(outdata_f, f'{ctry_code}_{cityLocalName_unicode}_GQA_centroids.shp')
    if not os.path.exists(output_path):
        print(f'loading aglomeration city {cityLocalName_unicode}')
        agl_city = agls.query(f'agglomerationName_localName == "{cityLocalName}"')
        # Keep only subset of columns
        agl_city = agl_city[['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry' ]]
        if agl_city.empty:
            agglomerationId_identifier = 'NotAvailable'
            print ("agglomerationId_identifier")
        else:
            agglomerationId_identifier = agl_city.agglomerationId_identifier.values.astype(str)[0]
            
            print ("agglomerationId_identifier")

            # Check noise contour maps GeoPackage file
            ncm_file_path = os.path.join(indata_f, 'NoiseData', f'Noise_20202025_export_{ctry_code}.gpkg')
            #layerName = f'dbo.DF48_agg_NoiseContours_roadsInAgglomeration_Lden_Valid_LatestDelivery_Poly_{ctry_code}'
            layerName = f'dbodf48_agg_noisecontours_roadsinagglomeration_lden_valid_latestdelivery_poly_{ctry_code}'
            ncm = gpd.read_file(ncm_file_path, layer=layerName)
            print ("ncm")

            # subset columns
            ncm_gdf = ncm[['category', 'geometry']]

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53726 instead
  warnings.warn(


Kaunas
2024-07-03 09:54:39.008198
loading aglomeration city Kaunas
agglomerationId_identifier
ncm


In [9]:
ncm_gdf['geometry'] = ncm_gdf['geometry'].simplify(tolerance=0.01)


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
uc_city['geometry'] = uc_city['geometry'].simplify(tolerance=0.01)

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [11]:
# Initialize Dask client
client = Client()


# Example usage
gdf1 = uc_city
gdf2 = ncm_gdf


# Ensure that GeoDataFrames are not empty
if gdf1.empty or gdf2.empty:
    print("One of the input GeoDataFrames is empty.")
else:
    try:
        result = overlay_with_dask(gdf1, gdf2, how='intersection', npartitions=10)
        result.to_file('path/to/result.shp')
        print("Overlay operation completed successfully.")
    except Exception as e:
        print(f"An error occurred during the overlay operation: {e}")

# Shut down the Dask client
client.close()


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53949 instead
  warnings.warn(


An error occurred during the overlay operation: 'DataFrame' object has no attribute 'overlay'


In [12]:
gdf1

,HDENS_CLST,HDENS_NAME,HDENS_2011,POPL_2021,CNTR_CODE,MBRS_CODE_,SHAPE_AREA,SHAPE_LEN,geometry
23,GEOSTAT21_051,Kaunas,GEOSTAT11_051,235793.0,LT,2,53000000.0,38000.0,"POLYGON ((5211000.000 3614000.000, 5210000.000..."


In [13]:
gdf2

,category,geometry
0,LdenLowerThan40,"POLYGON ((5215179.543 3621207.333, 5215177.736..."
1,LdenLowerThan40,"POLYGON ((5200261.108 3621747.573, 5200255.038..."
2,LdenLowerThan40,"POLYGON ((5211290.944 3612078.395, 5211290.802..."
3,LdenLowerThan40,"POLYGON ((5199699.320 3621473.095, 5199689.885..."
4,LdenLowerThan40,"POLYGON ((5216184.465 3613984.531, 5216178.006..."
...,...,...
15616,Lden4044,"POLYGON ((5208314.259 3620582.057, 5208303.904..."
15617,Lden4044,"POLYGON ((5209748.686 3613562.795, 5209740.408..."
15618,Lden4044,"POLYGON ((5209339.352 3617042.959, 5209335.758..."
15619,Lden4044,"POLYGON ((5217374.061 3622689.681, 5217378.165..."


In [20]:
# Example usage
gdf1 = uc_city
gdf2 = ncm_gdf

In [15]:
gdf1

,HDENS_CLST,HDENS_NAME,HDENS_2011,POPL_2021,CNTR_CODE,MBRS_CODE_,SHAPE_AREA,SHAPE_LEN,geometry
23,GEOSTAT21_051,Kaunas,GEOSTAT11_051,235793.0,LT,2,53000000.0,38000.0,"POLYGON ((5211000.000 3614000.000, 5210000.000..."


In [16]:
gdf1=gdf1.sindex

In [17]:
gdf1

In [18]:
gdf2=gdf2.sindex

In [19]:
dask_gdf1 = dgpd.from_geopandas(gdf1, npartitions=10)
dask_gdf2 = dgpd.from_geopandas(gdf2, npartitions=10)

# Perform the overlay operation
result = dask_gdf1.overlay(dask_gdf2, how=how).compute()

TypeError: Input must be a pandas DataFrame or Series.

In [21]:
# Example usage
gdf1 = uc_city
gdf2 = ncm_gdf
dask_gdf1 = dgpd.from_geopandas(gdf1, npartitions=10)
dask_gdf2 = dgpd.from_geopandas(gdf2, npartitions=10)

# Perform the overlay operation
result = dask_gdf1.overlay(dask_gdf2, how=how).compute()

AttributeError: 'DataFrame' object has no attribute 'overlay'

In [22]:
# Create spatial indices
sindex1 = (gdf1['geometry']).sindex
sindex2 = (gdf2['geometry']).sindex

In [23]:
dask_gdf1 = dgpd.from_geopandas(gdf1, npartitions=10)

In [24]:
dask_gdf1

,HDENS_CLST,HDENS_NAME,HDENS_2011,POPL_2021,CNTR_CODE,MBRS_CODE_,SHAPE_AREA,SHAPE_LEN,geometry
npartitions=1,,,,,,,,,
23,object,object,object,float64,object,int64,float64,float64,geometry
23,...,...,...,...,...,...,...,...,...


In [25]:
dask_gdf2

,category,geometry
npartitions=10,,
0,object,geometry
1563,...,...
...,...,...
14059,...,...
15620,...,...


In [27]:
result = dask_gdf1.overlay(dask_gdf2, how='intersect').compute()

AttributeError: 'DataFrame' object has no attribute 'overlay'

In [29]:
gdf1.dtypes

HDENS_CLST      object
HDENS_NAME      object
HDENS_2011      object
POPL_2021      float64
CNTR_CODE       object
MBRS_CODE_       int64
SHAPE_AREA     float64
SHAPE_LEN      float64
geometry      geometry
dtype: object

In [ ]:
dask_gdf1 = dgpd.from_geopandas(gdf1, npartitions=10)